In [3]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
import scipy.integrate as integrate

# DM power spectrum ran in Matlab

In [4]:
cdenPowsp_ml = np.loadtxt("cdenPowsp_omm_0_3_omv_0_7_ombh2_0_0_h_0_7_an_1_0_s8_0_8_ips_3.out", delimiter= " ")
cdenPowsp_ml

ValueError: could not convert string to float: 

In [ ]:
cdenPowsp_ml[:,1:2]

In [ ]:
plt.plot(cdenPowsp_ml[:,0:1], cdenPowsp_ml[:,1:2])
plt.xscale("log")
plt.yscale("log")

plt.ylabel("$P_m(k)$ [($h^{-1}$Mpc)$^3$]" )
plt.xlabel("wavelength $k$ [Mpc $h^{-1}$]" )

# CDM Power Spectrum

In [ ]:
#%
#%***********************************************************
#%*  [om_m,om_v,om_bh2,h,an,sigma8] = cdenCosparamInit; *
#%***********************************************************
#%***********************************************************
#% Initiates cosmological parameters.
#%
#% ARGUMENTS
#%
#% COMPATIBILITY: Matlab(?), Octave
#%
#% AUTHOR: Avery Meiksin
#%
#% HISTORY:
#%  10 10 17 Creation date.
#%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%
#function [om_m,om_v,om_bh2,h,an,sigma8] = cdenCosparamInit;
#global omega;
#%disp('Planck 2014');
#%om_m = 0.308;
#%om_v = 1 - om_m;
#%om_bh2 = 0.02214;
#%h = 0.678;
#%an = 0.9608;
#%sigma8 = 0.826;
#%
#%disp('Planck 2016');
#%om_m = 0.315;
#%om_v = 1 - om_m;
#%om_bh2 = 0.02222;
#%h = 0.6731;
#%an = 0.9655;
#%sigma8 = 0.829;
#%
#function [om_m,om_v,om_bh2,h,an,sigma8] = cdenCosparamInit;def cdenCosparamInit(om_m,om_v,om_bh2,h,an,sigma8)
print('Planck 2018')
om_m = 0.3111
om_v = 1 - om_m
om_bh2 = 0.02242
h = 0.6766
an = 0.9665
sigma8 = 0.8102
omega = (om_m, om_v, 0., om_bh2)
print(omega)

class Tk_all:
    def __init__(self,):
        self.tk = 1.0
        self.fk = 1.0 
        self.fkmin = 1.0
        self.fkmax = 1.0
Tk = Tk_all()

In [ ]:
#%
#%******************************************************
#%*  cdenFNorm(lgfk,pnorm,Gamma,an,Tk,xf,ismooth,ips)  *
#%******************************************************
#%******************************************************
#% Computes normalization integrand for power spectrum.
#%
#% ARGUMENTS
#%  lgfk       Array of (logarithmically-spaced) wavenumber.
#%  pnorm      Normalization of matter power spectrum.
#%  Gamma      Curvature of matter power spectrum.
#%  an         Spectral index of matter power spectrum.
#%  Tk         Transfer function for matter power spectrum.
#%  xf         Filtering scale (inverse units of dlfkp).
#%  ismooth    Method for smoothing fluctuations.
#%  ips        Method of computing matter power spectrum.
#%
#% COMPATIBILITY: Matlab(?), Octave
#%
#% REQUIREMENTS:
#%         cdenPowspInit.m, cdenPowsp.m
#%
#% AUTHOR: Avery Meiksin
#%
#% HISTORY:
#%  19 04 16 Creation date. (After FNORM subfunction in lss/src/Delta2.f.)
#%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%
#function FNORM = cdenFNorm(lgfk,pnorm,Gamma,an,Tk,xf,ismooth,ips);
def cdenFNorm(lgfk,pnorm=1,Gamma=1,an=1,Tk=1, xf=1, ismooth=1,ips=1):
    print('THIS IS lgfk',lgfk )
    fk = np.exp(lgfk)
    fk = (lgfk)
    #%fk = lgfk; %TEST for linear integration
    y = xf*fk
    if ismooth == 0:
        W = 1.0e0
    #end
    if ismooth == 1:
        W = 3.*(np.sin(y) - y*np.cos(y))/ y/ y/ y
    #end
    if ismooth == 2:
        W = 0.
        maskl = y < 10
        W[maskl]= 1./ np.exp(y[maskl]*y[maskl])
    #end
    if ismooth == 3:
        W = 1./ (1 + y*y)
    #end
    #print(lgfk,pnorm,Gamma,an,fk, fkmin,fkmax,Tk,ips)
    PS = cdenPowsp(lgfk ,pnorm,Gamma,an,Tk,ips)
    FNORM = fk*fk*fk*PS*W*W/ (2*np.pi**2)
    return FNORM
    #%FNORM = fk.*fk.*PS.*W.*W/ (2*pi^2); %TEST for linear integration


In [ ]:
# %
# %********************
# %*  PS = cdenPowsp(fk,pnorm,Gamma,an,Tk,ips)  *
# %********************
# %********************
# % Computes matter power spectrum.
# % Assumes k in Tk file is in conventional comoving h/ Mpc.
# % Conventionally pnorm is for P(k) in units (comoving Mpc/h)^3,
# % so PS is returned in units (comoving Mpc/h)^3.
# %
# % ARGUMENTS
# %  fkp        Array of wavenumbers.
# %  pnorm      Normalization of power spectrum.
# %  Gamma      Curvature of power spectrum.
# %  an         Tilt of power spectrum.
# %  Tk         Transfer function.
# %  ips        Method for computing power spectrum.
# %
# % COMPATIBILITY: Matlab(?), Octave
# %
# % AUTHOR: Avery Meiksin
# %
# % HISTORY:
# %  19 04 16 Creation date. (After lss/src/powsp.f.)
# %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %
#function PS = cdenPowsp(fk,pnorm,Gamma,an,Tk,ips);

def cdenPowsp(fk_i,pnorm=1,Gamma=1,an=1,Tk=1,ips=1):
    if ips == 1:
    #% use B&E
        a = 6.4/ Gamma
        b = 3.0/ Gamma
        c = 1.7/ Gamma
        PS = 1. + (fk_i*(a + c*c*fk_i) + (b*fk_i)**1.5)**1.13
        PS = pnorm*fk_i**an/ PS**(2./ 1.13)
    #end
    if ips == 2:
    #% use BBKS
        q = fk_i/ Gamma;
        PS = 1 + q*(3.89+q*(16.1*16.1+q*(5.46*5.46*5.46+q*6.71*6.71*6.71*6.71)))
        PS = np.log(1. + 2.34*q)/ PS**0.25/ (2.34*q)
        PS = pnorm*fk_i**an*PS*PS
    #end
    if ips == 3:
        print("DEBUG JUAN",Tk.fk.size,Tk.tk.size,fk_i)
        tki = np.interp(fk_i,Tk.fk,Tk.tk)

        PS = pnorm*(fk_i**an)*tki*tki
        maskl = fk_i < Tk.fkmin
        masku = fk_i > Tk.fkmax
        q = fk_i[maskl]/ Gamma
        PS[maskl] = 1 + q*(3.89+q*(16.1*16.1+q*(5.46*5.46*5.46+q*6.71*6.71*6.71*6.71)))
        PS[maskl] = np.log(1. + 2.34*q)/ PS[maskl]**0.25/ (2.34*q)
        PS[maskl] = pnorm*fk_i[maskl]**an*PS[maskl]*PS[maskl]
        #clear q;
        q = fk_i[masku]/ Gamma;
        PS[masku] = 1 + q*(3.89+q*(16.1*16.1+q*(5.46*5.46*5.46+q*6.71*6.71*6.71*6.71)))
        PS[masku] = np.log(1. + 2.34*q)/ PS[masku]**0.25/ (2.34*q)
        PS[masku] = pnorm*fk_i[masku]**an*PS[masku]*PS[masku]
    return PS
    #end



In [ ]:
#%
#%********************
#%*  [pnorm,Gamma,Tk] = cdenPowspInitFile(om_m,om_v,om_bh2,h,an,sigma8,ips,Tk_file,nrows)  *
#%********************
#%********************
#% Initialise matter power spectrum computation using input file list of Tk files.
#% Assumes k in Tk file is in conventional comoving h/ Mpc.
#% Returned pnorm is for P(k) in units (comoving Mpc/h)^3.
#%
#% ARGUMENTS
#%  om_m       Total mass parameter.
#%  om_v       Vacuum energy parameter.
#%  om_bh2     Baryonic mass parameter x h^2.
#%  h          Hubble constant H0/ (100 km/s/Mpc).
#%  an         Tilt of matter power spectrum.
#%  sigma8     Sigma on 8 Mpc/h scale to normalize power spectrum to.
#%  ips        Method for computing power spectrum.
#%             ips = 1: B&E; ips = 2: BBKS; ips = 3: Tk file
#%  Tk_file    T(k) file for ips = 3
#%  nrows      Number of rows in T(k) file
#%
#% OUTPUT
#%  pnorm      Normalization of power spectrum  ([cMpc/h]^3).
#%  Gamma      Curvature of power spectrum.
#%  Tk         Transfer function.
#%
#% COMPATIBILITY: Matlab(?), Octave
#%
#% AUTHOR: Avery Meiksin
#%
#% HISTORY:
#%  28 11 17 Creation date. (From cdenPowspInit.m.)
#%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%

def cdenPowspInitFile(om_m,om_v,om_bh2,h,an,sigma8,ips,Tk_file,nrows):
    #%function [pnorm,Gamma,Tk] = cdenPowspInitFile(om_m,om_v,om_bh2,h,an,sigma8,ips,Tk_file,nrows);
    #%ng = 8388608;
    ng = 1e6
    boxsize = 1000
    tol = 1e-6
    fkl = 2*np.pi/ boxsize
    fku = 0.5*ng*fkl
    lgfkl = np.log(fkl)
    lgfku = np.log(fku)
    #%lgfkl = fkl %TEST for linear integration
    #%lgfku = fku %TEST for linear integration
    #%Uses Bunn & White (1997) COBE normalization.
    tilt = an - 1
    om_b = om_bh2/ (h*h)
    #%%%%%%%%%%%%%%%%%%%%%
    if om_v > 0:
        dh=1.94e-5*(om_m**(-0.785-0.05*np.log(om_m)))*np.exp(-0.95*tilt-0.169*tilt*tilt)
    else:
        dh=1.95e-5*(om_m**(-0.35-0.19*np.log(om_m)-0.17*tilt))*np.exp(-tilt-0.14*tilt*tilt)
    #%end
    #%
    pnorm = 2*np.pi*np.pi*dh*dh*(2997.9)**(3+an)
    Gamma = om_m*h/ (np.exp(om_b*(1 + 1./ om_m)))
    if ips == 3:
        #%  Tk_file = input('Enter name of Tk file: ','s');
        #%  nr = input('Enter number of rows: ');
        #%%%fid = fopen(Tk_file,'r');
        Tk.fk,Tk.tk = np.loadtxt(Tk_file,unpack=True)
        Tk.fkmin = np.min(fk)
        Tk.fkmax = np.max(fk)
        #
        #fid = open(Tk_file,'r',encoding = 'utf-8')
        ##Tk_table = fscanf(fid,'%g %g',[2,nrows]);
        #fid.read()
        #Tk_table = Tk_table';  
        #Tk.fk = Tk_table(:,1);
        #Tk.fkmin = min(Tk.fk);
        #Tk.fkmax = max(Tk.fk);
        #Tk.tk = Tk_table(:,2);
    else:
        Tk.tk = 0
        Tk.fk = 0
        Tk.fkmin = 0
        Tk.fkmax = 0
    print('Bunn & White COBE pnorm: ')
    print(pnorm)
    #% Now normalize to desired sigma8.
    xf = 8
    ismooth = 1 #%spherical top hat
    print("Integration limits: ",lgfkl,lgfku)
    #sigma8_BW = integrate.quad(lambda x: cdenFNorm(x,pnorm=pnorm,Gamma=Gamma,
    #                                     an=an,Tk=Tk, xf=xf,ismooth=ismooth,ips=ips),lgfkl,lgfku)
    logos = np.logspace(-5.0698782125727915, 8.052485164831538,10000)
    sigma8_BW = integrate.simps(cdenFNorm(logos,pnorm=pnorm,Gamma=Gamma,
                                         an=an,Tk=Tk, xf=xf,ismooth=ismooth,ips=ips),logos)
    #%% TEST for linear integration
    #%fk=linspace(lgfkl,lgfku,1000000);
    #%fk(1)
    #%fk(1000000)
    #%dk = (lgfku-lgfkl)/1000000;
    #%u = cdenFNorm(fk,pnorm,Gamma,an,Tk,xf,ismooth,ips);
    #%sigma8_BW = dk*trapz(u);
    #%%
    sigma8_BW = sigma8_BW**0.5;
    pnorm = pnorm*(sigma8/ sigma8_BW)*(sigma8/ sigma8_BW)
    #lgfk,pnorm,Gamma,an,fk, fkmin, fkmax, tk, xf, ismooth,ips
    #print(lgfk)
    return pnorm, Gamma, Tk
        
    

In [ ]:
pwd

In [ ]:


fk = 10**np.linspace(-4,-1,31)
#[om_m,om_v,om_bh2,h,an,sigma8] = cdenCosparamInit;
ips = 3
Tk_file = '/Users/teresita/Documents/cosmic reionization paper/DarkMatterPowerSpectrum/tkO3111L6889h6766N2242_eh_PLANCK18.dat'
nrows = 650

#% Convert to strings
#som_m = str(om_m,'%.1f')
#som_v = str(om_v,'%.1f')
#som_bh2 = str(om_bh2,'%.1f')
#sh = str(h,'%.1f')
#san = str(an,'%.1f')
#ssigma8 = str(sigma8,'%.1f')
#sips = str(ips,'%i')

som_m = str(om_m)
som_v = str(om_v)
som_bh2 = str(om_bh2)
sh = str(h)
san = str(an)
ssigma8 = str(sigma8)
sips = str(ips)



#suffix = '_omm_'
#suffix = strcat(suffix,som_m) 
#suffix = strcat(suffix,'_omv_')
#suffix = strcat(suffix,som_v)
#suffix = strcat(suffix,'_ombh2_')
#suffix = strcat(suffix,som_bh2)
#suffix = strcat(suffix,'_h_')
#suffix = strcat(suffix,sh)
#suffix = strcat(suffix,'_an_')
#suffix = strcat(suffix,san)
#suffix = strcat(suffix,'_s8_')
#suffix = strcat(suffix,ssigma8)
#suffix = strcat(suffix,'_ips_')
#suffix = strcat(suffix,sips)
#prefix = 'cdenPowsp'

outfile_name = 'cdenPowsp_omm_'+ som_m + '_omv_' + som_v + '_ombh2_' + som_bh2 + '_h_' + sh + '_an_' + san + '_s8_' + ssigma8 + '_ips_' + sips + '.out'

#matoutfile = (outfile,'.txt')
#outfile = strcat(outfile,'.out')

#pnorm,Gamma,tk = cdenPowspInitFile(om_m,om_v,om_bh2,h,an,sigma8, ips,Tk_file,nrows)
pnorm, Gamma, tk = cdenPowspInitFile(om_m,om_v,om_bh2,h,an,sigma8, ips,Tk_file,nrows)
PS = cdenPowsp(fk,pnorm,Gamma,an,tk,ips)
lenfk = len(fk)
Aout = (fk[:],PS[:])
#fido = fopen(outfile,'w')
#print(fido,'%8.6f %16.6e\n',Aout)
#fclose(fido)

#save(matoutfile, 'fk', 'PS', 'om_m', 'om_v','om_bh2', 'h', 'an', 'sigma8', 'ips','Tk_file')


In [ ]:
plt.plot(fk,PS, ls = '--')
plt.plot(cdenPowsp_ml[:,0:1], cdenPowsp_ml[:,1:2])
plt.xscale("log")
plt.yscale("log")

In [ ]:
PS

In [ ]:
ls